In [1]:
import tensorflow as tf
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator

## install the following library to help split the dataset into train and test

In [4]:
# !pip install split-folders

In [6]:
# UNCOMMENT THIS FOR THE FIRST TIME TO MAKE THOSE SPLITS BUT NOT NEEDED AFTER
#import splitfolders
# Split the dataset into train (80%) and test (20%)
#splitfolders.ratio("flower_images", output="flowers_split", seed=1337, ratio=(.8, .2))

In [8]:
train_datagen = ImageDataGenerator(rescale=1./255,
    rotation_range=40,
    zoom_range=0.2,
    horizontal_flip=True,
    shear_range=0.2,
    width_shift_range=0.2,
    height_shift_range=0.2,
    fill_mode='nearest')
val_datagen = ImageDataGenerator(rescale=1./255)

In [10]:
train_generator = train_datagen.flow_from_directory(
    'flowers_split/train',
    target_size=(128, 128),
    batch_size=32,
    class_mode='categorical'
)

Found 4008 images belonging to 5 classes.


In [12]:
val_generator = val_datagen.flow_from_directory(
    'flowers_split/val',
    target_size=(128, 128),
    batch_size=32,
    class_mode='categorical',
    shuffle=False
)

Found 1002 images belonging to 5 classes.


In [14]:
train_generator.class_indices

{'Lilly': 0, 'Lotus': 1, 'Orchid': 2, 'Sunflower': 3, 'Tulip': 4}

### if class indice create a hidden zero index ".ipynb_checkpoints", you can remove it using your CMD to remain with 5 classes instead of 6 using the following commented command in your CMD IF YOU USE WINDOWS

In [17]:
# for /d /r %i in (*.ipynb_checkpoints) do @if exist "%i" rd /s /q "%i"

# Building CNN

In [20]:
cnn = tf.keras.models.Sequential()

# first layer

In [23]:
cnn.add(Conv2D(filters=32, kernel_size=3, activation='relu', input_shape=(128, 128, 3)))
cnn.add(MaxPooling2D(pool_size=2, strides=2))

C:\Users\user\anaconda3\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


# second layer


In [26]:
cnn.add(Conv2D(filters=64, kernel_size=3, activation='relu'))
cnn.add(MaxPooling2D(pool_size=2, strides=2))

# third Layer

In [29]:
cnn.add(Conv2D(filters=128, kernel_size=3, activation='relu'))
cnn.add(MaxPooling2D(pool_size=2, strides=2))

# Flatten + Dense + Dropout

In [32]:
cnn.add(Flatten())
cnn.add(Dense(units=128, activation='relu'))
cnn.add(Dropout(0.5))
cnn.add(Dense(units=5, activation='softmax'))

# compiling

In [35]:
cnn.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

# Train with Early Stopping

In [38]:
from tensorflow.keras.callbacks import EarlyStopping

early_stop = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

In [40]:
cnn.fit(train_generator, validation_data=val_generator, epochs=20, callbacks=[early_stop])

C:\Users\user\anaconda3\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/20
126/126 ━━━━━━━━━━━━━━━━━━━━ 128s 981ms/step - accuracy: 0.3653 - loss: 1.4674 - val_accuracy: 0.5230 - val_loss: 1.1854
Epoch 2/20
126/126 ━━━━━━━━━━━━━━━━━━━━ 138s 949ms/step - accuracy: 0.5150 - loss: 1.2112 - val_accuracy: 0.5569 - val_loss: 1.0959
Epoch 3/20
126/126 ━━━━━━━━━━━━━━━━━━━━ 133s 880ms/step - accuracy: 0.5575 - loss: 1.1308 - val_accuracy: 0.5828 - val_loss: 1.0451
Epoch 4/20
126/126 ━━━━━━━━━━━━━━━━━━━━ 107s 850ms/step - accuracy: 0.5799 - loss: 1.0719 - val_accuracy: 0.5818 - val_loss: 1.0213
Epoch 5/20
126/126 ━━━━━━━━━━━━━━━━━━━━ 110s 876ms/step - accuracy: 0.5664 - loss: 1.0933 - val_accuracy: 0.6168 - val_loss: 0.9706
Epoch 6/20
126/126 ━━━━━━━━━━━━━━━━━━━━ 109s 864ms/step - accuracy: 0.6069 - loss: 1.0064 - val_accuracy: 0.6218 - val_loss: 0.9439
Epoch 7/20
126/126 ━━━━━━━━━━━━━━━━━━━━ 110s 866ms/step - accuracy: 0.6205 - loss: 0.9763 - val_accuracy: 0.6168 - val_loss: 0.9078
Epoch 8/20
126/126 ━━━━━━━━━━━━━━━━━━━━ 112s 889ms/step - accuracy: 0.6380 -

In [42]:
from sklearn.metrics import classification_report, confusion_matrix
import numpy as np
# Get true labels
true_labels = val_generator.classes

# Get class indices
class_indices = val_generator.class_indices
labels = list(class_indices.keys())

# Get predictions
predictions = cnn.predict(val_generator)
predicted_classes = np.argmax(predictions, axis=1)

32/32 ━━━━━━━━━━━━━━━━━━━━ 12s 359ms/step


In [43]:
# Print classification report
print("Classification Report:")
print(classification_report(true_labels, predicted_classes, target_names=labels))

# Optional: Confusion Matrix
print("Confusion Matrix:")
print(confusion_matrix(true_labels, predicted_classes))


Classification Report:
              precision    recall  f1-score   support

       Lilly       0.61      0.70      0.66       200
       Lotus       0.73      0.75      0.74       202
      Orchid       0.73      0.67      0.70       200
   Sunflower       0.90      0.97      0.94       200
       Tulip       0.81      0.66      0.73       200

    accuracy                           0.75      1002
   macro avg       0.76      0.75      0.75      1002
weighted avg       0.76      0.75      0.75      1002

Confusion Matrix:
[[141  19  19   8  13]
 [ 26 152  14   0  10]
 [ 38  17 134   3   8]
 [  4   1   0 195   0]
 [ 21  20  17  10 132]]


In [46]:
import numpy as np
from tensorflow.keras.preprocessing import image

# Load and preprocess the image
test_image = image.load_img('testFlower.jpeg', target_size=(128, 128))
test_image = image.img_to_array(test_image)
test_image = np.expand_dims(test_image, axis=0)
test_image = test_image / 255.0

result = cnn.predict(test_image)
class_indices = {
    0: 'Lilly',
    1: 'Lotus',
    2: 'Orchid',
    3: 'Sunflower',
    4: 'Tulip'
}

predicted_index = np.argmax(result)
prediction = class_indices[predicted_index]

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 271ms/step


In [48]:
print(f"The model predict this image to be : {prediction}")

The model predict this image to be : Sunflower


In [50]:
# !pip install gradio

In [52]:
cnn.save("modelFlowers.keras")

In [54]:
import gradio as gr
import numpy as np
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image

# Load your trained model
model = load_model("modelFlowers.keras") 

# Define your class names
class_indices = {
    0: 'Lilly',
    1: 'Lotus',
    2: 'Orchid',
    3: 'Sunflower',
    4: 'Tulip'
}

# Define prediction function
def predict_flower(img):
    img = img.resize((128, 128))  # Resize to model input size
    img = image.img_to_array(img)
    img = np.expand_dims(img, axis=0)
    img = img / 255.0
    prediction = model.predict(img)
    index = np.argmax(prediction)
    return class_indices[index]

# Gradio interface
iface = gr.Interface(
    fn=predict_flower,
    inputs=gr.Image(type="pil"),
    outputs="text",
    title="Flower Image Classifier",
    description="Upload a flower image and get its predicted class"
)

# Launch inline in notebook
iface.launch(share=False, inline=True)

* Running on local URL:  http://127.0.0.1:7861

To create a public link, set `share=True` in `launch()`.
